In [1]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join('../../..')))

from Notebooks.setupConfig import setup_config
setup_config()

FUNCTIONS_WORKER_RUNTIME: python
pythonPath: C:\Users\Shubham Vishwakarma\AppData\Local\Microsoft\WindowsApps\python.exe
APPINSIGHTS_INSTRUMENTATIONKEY: f0cd5280-7f75-4c2e-9e92-93bc75b8a8c4
APPINSIGHTS_ConnectionString: InstrumentationKey=f0cd5280-7f75-4c2e-9e92-93bc75b8a8c4
KEY_VAULT_URI: https://kv-tradinglead.vault.azure.net/
TelemetryPropApp: TradingLead - Backend Function App
TelemetryPropEnv: local
TelemetryPropVersion: 1.0.0
TelemetryPropService: TradingLead - General Service
SCHEDULE_CRON_FOR_ACCESS_TOKEN_GENERATOR: 0 * * * * *
SCHEDULE_CRON_FOR_PARTICIPATION_DATA_STORE: 0 */10 * * * *
NumberOfDataParticipationDataToBeFetched: 20


In [3]:
import datetime
import azure.functions as func
from SharedCode.Repository.Logger.logger_service import LoggerService
from SharedCode.Utils.constants import Constants
from SharedCode.Utils.utility import FunctionUtils
from TimerTriggerForAccessToken.access_token_generator_runner import access_token_generator_runner

Failed to integrate module: opencensus.ext.logging.trace
No module named 'opencensus.ext.logging'


In [4]:
operation_id = "RandomOperationId"

tel_props = {
        Constants.SERVICE : Constants.access_token_generator_service,
        Constants.operation_id : operation_id,
    }

In [ ]:
results = access_token_generator_runner(tel_props)

In [ ]:
results

In [ ]:
from SharedCode.Repository.AccessToken.access_token import get_fyers_access_token
import json
import time
from SharedCode.Repository.KeyVault.keyvault_service import KeyVaultService
from SharedCode.Repository.Logger.logger_service import LoggerService
from SharedCode.Repository.Cache.redis_cache_service import RedisCacheService
from SharedCode.Utils.constants import Constants
import concurrent.futures


kv_service = KeyVaultService()
telemetry = LoggerService()
redis_cache_service = RedisCacheService()

In [ ]:
fyer_users_json = kv_service.get_secret("FyerUserDetails")
telemetry.info(fyer_users_json, tel_props)
fyer_users = json.loads(fyer_users_json)
results = []

In [ ]:

def fetch_and_store_token(user, tel_props):
    max_retries = 3
    for attempt in range(max_retries):
        try:
            fyers_details_json = kv_service.get_secret(user['KvSecretName'])
            fyers_details = json.loads(fyers_details_json)
            username = fyers_details[Constants.fyers_username]

            tel_props.update( {
                Constants.username : username,
                Constants.client_id : fyers_details[Constants.client_id],
                Constants.contact_number : fyers_details[Constants.contact_number],
            })
            redis_key = f"{username}-token"
            access_token = get_fyers_access_token(fyers_details, tel_props)
            redis_cache_service.set_value(redis_key, access_token)
            fetched_token = redis_cache_service.get_value(redis_key)
            
            telemetry.info(f"Token stored for {username}, {fetched_token}", tel_props)
            return {"user": user, "fetched_token": fetched_token}

        except Exception as e:
            telemetry.exception(f"Error in fetching access token attempt no. {attempt}.\nError: {e}", tel_props)
            time.sleep(10)
            continue
    message = f"Max retries in main function. Error in fetching access token. for user {username}"
    telemetry.exception(message, tel_props)
    raise Exception(message)

In [ ]:
fetch_and_store_token(fyer_users[1], tel_props)

In [ ]:
fyer_users

In [ ]:
fyers_details_json = kv_service.get_secret(fyer_users[1]['KvSecretName'])

In [ ]:
fyers_details = json.loads(fyers_details_json)
username = fyers_details[Constants.fyers_username]

tel_props.update( {
    Constants.username : username,
    Constants.client_id : fyers_details[Constants.client_id],
    Constants.contact_number : fyers_details[Constants.contact_number],
})

In [ ]:
# fyers_details["ClientId"] = 'EZ0JD38O15-100'
fyers_details

In [ ]:
access_token = get_fyers_access_token(fyers_details, tel_props)

In [ ]:
fyer_users

In [ ]:
redis_key = "XJ13618-token"

In [ ]:
redis_cache_service.get_value(redis_key)